<a href="https://colab.research.google.com/github/burakbudanur/llmdocstring/blob/master/server/gendocstring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install flask pyngrok huggingface_hub

In [2]:
# uncomment and run with your own authtoken
# !ngrok config add-authtoken <authtoken>

In [3]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python # gpu available
# !CMAKE_ARGS="-DLLAMA_BLAS=ON -DLLAMA_BLAS_VENDOR=OpenBLAS" pip install llama-cpp-python # cpu only

In [4]:
import os
import threading
from datetime import datetime
from flask import Flask, jsonify, request
from pyngrok import ngrok
import time
import json
from pathlib import Path
from huggingface_hub import hf_hub_download
from llama_cpp import Llama
import multiprocessing
import re

num_cpu = multiprocessing.cpu_count()
print(num_cpu)

2


In [5]:
def log(string):

    # datetime object containing current date and time
    now = datetime.now()

    print("now =", now)
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")

    f = open("ngrokweb.log", "a")
    f.write(dt_string + '\n')
    f.close()

    with open("ngrokweb.log","a") as f:
        f.writelines(string)

In [6]:
def download_weights():

    weights_dir = Path("weights")
    weights_dir.mkdir(exist_ok=True)

    repo_id = "TheBloke/Mistral-7B-Instruct-v0.1-GGUF"
    filename = "mistral-7b-instruct-v0.1.Q6_K.gguf"

    if (weights_dir / Path(filename)).exists():
        print(
            f"{filename} exists. Delete manually if you wish to download again."
            )
        return
    else:
        return hf_hub_download(
            repo_id = repo_id, filename=filename, local_dir='./weights/',
            local_dir_use_symlinks = False
            )

download_weights()

mistral-7b-instruct-v0.1.Q6_K.gguf exists. Delete manually if you wish to download again.


In [7]:
model = "weights/mistral-7b-instruct-v0.1.Q6_K.gguf"  # instruction model
llm = Llama(
    model_path=model, n_ctx=8192, n_batch=128, n_threads=num_cpu,
    n_gpu_layers=-1, verbose=True, seed=42
    )

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [8]:
def get_docstring_from_template(code, template):

    instruction = "According to the template, produce a docstring for the "
    instruction += "following python function."
    instruction += "Return the full function definition with the docstring."

    message = f"Docstring template: \n {template} \n <s>[INST] {instruction} [/INST]</s> \n {code}"
    output = llm(message, echo=True, stream=False, max_tokens=4096)
    text = output['choices'][0]['text']
    print(text)
    docstring_pattern = re.compile(r'\'\'\'(.*?)\'\'\'|\"\"\"(.*?)\"\"\"', re.DOTALL)
    match = docstring_pattern.search(text)

    return match.groups()[1] if match else "Docstring generation failed, please try again."

In [9]:
template = """
    [Summary of the function fibonacci_of]

    Args:
        n ([type]): [description]

    Returns:
        [type]: [description]
    """

code = """
def fibonacci_of(n):
    if n in cache:
        return cache[n]
    cache[n] = fibonacci_of(n - 1) + fibonacci_of(n - 2)
    return cache[n]
"""

docstr_template = get_docstring_from_template(code, template)

Docstring template: 
 
    [Summary of the function fibonacci_of]

    Args:
        n ([type]): [description]

    Returns:
        [type]: [description]
     
 <s>[INST] According to the template, produce a docstring for the following python function.Return the full function definition with the docstring. [/INST]</s> 
 
def fibonacci_of(n):
    if n in cache:
        return cache[n]
    cache[n] = fibonacci_of(n - 1) + fibonacci_of(n - 2)
    return cache[n]

"""
Summary of the function fibonacci_of

Args:
    n (int): The position for which to return the fibonacci number.

Returns:
    int: The Nth Fibonacci number, where 0 = 0 and 1 = 1
     
"""


In [10]:
print(docstr_template)


Summary of the function fibonacci_of

Args:
    n (int): The position for which to return the fibonacci number.

Returns:
    int: The Nth Fibonacci number, where 0 = 0 and 1 = 1
     



In [11]:
app = Flask(__name__)
port = "5000"

# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(port).public_url
print(
    " * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port)
    )

# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url

# ... Update inbound traffic via APIs to use the public-facing ngrok URL


# Define Flask routes
@app.route("/")
def index():
    return "Hello from llmdocstring server."

@app.route("/summary", methods=["POST"])
def summary():
    if request.method == "POST":
        payload = request.get_json()
        t0 = time.time()

        # Generate docstring here

        code = payload["code"]
        snippet = payload["snippet"]
        template = snippet.replace('\"\"\"\n', '')

        log("code:")
        log(code)
        log("snippet:")
        log(snippet)

        docstring = get_docstring_from_template(code, template)
        if docstring[0:1] == '\n':
            docstring = docstring[1:]
        docstring = docstring.rstrip()+'\n'

        log(docstring)

        t1 = time.time()
        result = {
            'message' : [docstring],
            'time' : (t1 - t0),
            'device' : "computer",
            'length' : len(docstring)
        }

        return jsonify(**result)

# Start the Flask server in a new thread
threading.Thread(target=app.run, kwargs={"use_reloader": False}).start()

 * ngrok tunnel "https://f8bf-34-105-6-148.ngrok-free.app" -> "http://127.0.0.1:5000"
